# Roles: Multi-label Single-class Classifier

In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

## Loading & Preprocessing of Labels

In [2]:
train = pd.read_csv('r.csv')    # reading the csv file
train.head(2)      # printing first five rows of the file

,Annotations,Path,Manuscript,Description,label,id,saint,prophet,king
0,"['initiale historiee', 'lettre f', 'lettre a p...",1366264.jpg,"Paris, Bibl. Mazarine, 0018",saint jerome ecrivant,['saint'],['1366264'],1,0,0
1,"['initiale historiee', 'lettre f', 'lettre a p...",1366263.jpg,"Paris, Bibl. Mazarine, 0018",saint jerome ecrivant,['saint'],['1366263'],1,0,0


In [3]:
train['label'][2]

"['prophet']"

In [4]:
y = np.array(train.drop(['Annotations','Path','Manuscript','Description','id','saint','prophet','king'],axis=1))
y.shape

(1791, 1)

In [5]:
# Converts a class vector (integers) to binary class matrix
# y = tf.keras.utils.to_categorical(y)

In [6]:
print(y.shape)
print(y)

(1791, 1)
[["['saint']"]
 ["['saint']"]
 ["['prophet']"]
 ...
 ["['king']"]
 ["['king']"]
 ["['king']"]]


## Loading & Preprocessing of Images

In [7]:
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [8]:
train_image = []
for i in tqdm(range(train.shape[0])):
    #"C:/Users/yhawa/OneDrive/masters_thesis_2022/Initiale_clean/hand_gestures/filtered_imageshand_gestures_new/"
    img = image.load_img('C:/Users/yhawa/OneDrive/masters_thesis_2022/Initiale_clean/roles/personspersons_new/'+train['Path'][i],target_size=(400,400,3))
    img = image.img_to_array(img)
    #reshape images
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 1791/1791 [01:25<00:00, 20.99it/s]


## Split Train and Test Data 80/20

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [10]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1432, 400, 400, 3) (1432, 1)
(359, 400, 400, 3) (359, 1)


## Model Architecture

In [11]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 396, 396, 16)      1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 198, 198, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 198, 198, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 194, 194, 32)      12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 97, 97, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 97, 97, 32)        0

## Model Training

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# X_train

In [15]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: Shapes (None, 1) and (None, 3) are incompatible